In [1]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 42kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=e621c0728ccc7684feaaac8e6c5719fe47bb807a211e9d12156d876692337a48
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [2]:
# Mount to google drive 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd /content/drive/My\ Drive/DeepLearning/FaceNet\ -\ Practice

/content/drive/My Drive/DeepLearning/FaceNet - Practice


In [4]:
import os 
from face_recognition import load_image_file, face_encodings, face_locations, compare_faces

DATABASE = "./data"

names = []
known_faces = []
for name in os.listdir(DATABASE):
  for filename in os.listdir(f"{DATABASE}/{name}"):
    img = load_image_file(f"{DATABASE}/{name}/{filename}")
    # Face detection works so good with this data view from the front 
    locations = face_locations(img)
    # Get the largest face in a image
    locations.sort(key=lambda x:(x[1]-x[3])*(x[2]-x[0]))
    # print(name+filename, locations[-1])
    if len(locations) == 0: continue
    face_enc = face_encodings(img, [locations[-1]])[0]
    known_faces.append(face_enc)
    names.append(name)

In [5]:
!pip install face_detection

In [ ]:
import tensorflow as tf
import face_detection
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import os 
 
# TOLERANCE = 0.7

TOLERANCES = [0.4, 0.45, 0.5, 0.55, 0.6]
for n_video, TOLERANCE in enumerate(TOLERANCES):

  count = 0
  cap = cv2.VideoCapture('./test.mp4')
  detector = face_detection.build_detector(
          "DSFDDetector",
          max_resolution=1080
      )
  fourcc = cv2.VideoWriter_fourcc(*'MP4V')
  out = cv2.VideoWriter(f'out_{n_video}.mp4', fourcc, 30.0, (1280, 720))
  labels = ["Tran Thanh", "Truong Giang", "Lan Ngoc", "Kha Nhu", 
            "Mac Van Khoa", "Anh Tu"]

  model = "cnn"
  names = np.array(names)

  while(True):
      # Capture frame-by-frame
      count += 1 
      ret, im = cap.read()
      if not ret:
        break
      t = time.time()

      face_loc = face_locations(im, model=model)
      encoding_faces = face_encodings(im, face_loc)
      print(f"Detection time: {time.time()- t:.3f}")

      for face_encoding, face_location in zip(encoding_faces, face_loc):
        results = compare_faces(known_faces, face_encoding, TOLERANCE)
        (top, right, bottom, left) = face_location 
        name = "Unknown"
        results = np.array(results)
        tmp = 0
        if True in results:
          count_class = np.zeros(6, dtype=int)
          for class_name in names[np.where(results == True)[0]]:
            count_class[labels.index(class_name)] += 1
          tmp = count_class.max()
          # Threshold number of image as same as test image must be
          #     greater than 12 (about ~30 images each class)
          if tmp >= 12:
            name = labels[count_class.argmax()] 

        cv2.rectangle(im, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(im, f'{name}-{tmp}', (left, top), cv2.FONT_HERSHEY_SIMPLEX,  
                    1, (255, 0, 0), 2, cv2.LINE_AA)
      out.write(im)
      print(f"Recognition time: {time.time()- t:.3f}")


  # When everything done, release the capture
  cap.release()
  out.release()
  cv2.destroyAllWindows()